In [1]:
__author__ = "Alec Brickner"

This is just a space to play around with the ideas from Chung and Bowman 2017. We'll be using the SST for this in order to see how the ideas in this paper work out on other datasets.

In [2]:
from collections import Counter
from rnn_classifier import RNNClassifier
from sklearn.linear_model import LogisticRegression
import sst
import tensorflow as tf
from tf_rnn_classifier import TfRNNClassifier
from tree_nn import TreeNN
import nltk

C:\Users\Alec\Anaconda3\envs\nlu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import tf_lifted_trnn

In [4]:
import importlib
importlib.reload(tf_lifted_trnn)

<module 'tf_lifted_trnn' from 'C:\\Users\\Alec\\Documents\\CS224U\\cs224u\\tf_lifted_trnn.py'>

First, let's make sure that we can get the model to compile.

In [246]:
def fit_tf_trnn_classifier(X, y):
    vocab = sst.get_vocab(map(lambda x: x.leaves(), X), n_words=3000)
    mod = tf_lifted_trnn.TfTreeRNNClassifier(
        vocab, 
        eta=0.05,
        batch_size=2048,
        embed_dim=50,
        max_length=200, 
        max_iter=10,
        cell_class=tf.nn.rnn_cell.LSTMCell,
        hidden_activation=tf.nn.tanh,
        train_embedding=True)
    mod.fit(X, y)
    return mod

Each example is represented as a tree. We need to convert this into a form that's nice for our Tensorflow RNNs. It would do us well to feed in:
- The sequence of words
- The tree structure
But how can we feed in the tree structure in vector form?
According to https://github.com/erickrf/treernn, we're going to want to make the graph static in order to greatly speed up computation. We want the vector representation that best works with the graph construction.

The above link has a pretty neat method for storing trees as lists. The idea is that we build the tree from the bottom-up, left to right. Though, the concept of combining nodes that are next to each other might be better. I'll play around with some ideas.

In [6]:
reader = sst.train_reader()

In [7]:
tree, score = next(reader)

In [8]:
tree2, score2 = next(reader)

In [9]:
x = [subtree for subtree in tree]

In [10]:
x[0][0]

LookupError: 

===========================================================================
NLTK was unable to find the gs file!
Use software specific configuration paramaters or set the PATH environment variable.
===========================================================================

Tree('2', ['The'])

In [244]:
import importlib
importlib.reload(tf_lifted_trnn)
importlib.reload(sst)

<module 'sst' from 'C:\\Users\\Alec\\Documents\\CS224U\\cs224u\\sst.py'>

In [228]:
vocab = sst.get_vocab([tree.leaves(), tree2.leaves()], n_words=3000)
mod = tf_lifted_trnn.TfTreeRNNClassifier(
    vocab, 
    eta=0.05,
    batch_size=2048,
    embed_dim=50,
    max_length=100, 
    max_iter=500,
    cell_class=tf.nn.rnn_cell.LSTMCell,
    hidden_activation=tf.nn.tanh,
    train_embedding=True)

In [229]:
mod.fit([tree, tree2], [score, score2])

Iteration 500: loss: 0.6931471824645996

In [ ]:
_ = sst.experiment(
    lambda x: x,                      # Free to write your own!
    fit_tf_trnn_classifier,             # Free to write your own!
    vectorize=False,
    train_reader=sst.train_reader,     # Fixed by the competition.
    assess_reader=sst.dev_reader,      # Fixed.
    class_func=sst.binary_class_func)  # Fixed.